In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


### CommaSeparatedListOutputParser

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
pprint(prompt.partial_variables) # 출력 형식 지침

{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}


In [3]:

# OpenAI 모델 설정
# model = ChatOpenAI(temperature=0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": "AI 관련 기술"})

# 쉼표로 구분된 리스트 출력
print(" AI 관련 기술 목록:")
print(result)

# 결과 활용 예시: CSV 파일로 저장
csv_filename = "./data/ai_technologies.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["AI 기술"])  # 헤더 추가
    for item in result:
        writer.writerow([item])

print(f" '{csv_filename}' 파일로 저장 완료!")


 AI 관련 기술 목록:
['Machine Learning', 'Deep Learning', 'Natural Language Processing', 'Computer Vision', 'Robotics']
 './data/ai_technologies.csv' 파일로 저장 완료!


### JsonOutputParser

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
print(prompt)

input_variables=['question'] input_types={} partial_variables={'format_instructions': 'Return a JSON object.'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='#Format: {format_instructions}\n\n#Question: {question}'), additional_kwargs={})]


In [5]:

# OpenAI 모델 설정
# model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))


[
    {
        "mission_name": "뉴호라이즌스",
        "goal": "명왕성 탐사",
        "agency": "NASA"
    },
    {
        "mission_name": "카시니-호이겐스",
        "goal": "토성의 위성 타이탄 탐사",
        "agency": "NASA, ESA, 이탈리아 우주국"
    },
    {
        "mission_name": "창어 4호",
        "goal": "달의 뒷면 탐사",
        "agency": "중국 우주국"
    }
]


### PandasDataFrameOutputParser

In [6]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Titanic 데이터셋 로드
df = pd.read_csv('data/titanic.csv')

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)
# print(parser)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
# print("Format Instructions: \n", format_instructions)


In [7]:
# ChatOpenAI 모델 초기화
# model = ChatOpenAI(temperature=0, model_name="gpt-4o")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)
print(prompt.partial_variables['columns'])

Survived, Pclass, Name, Sex, Age, Siblings/Spouses Aboard, Parents/Children Aboard, Fare


In [8]:
# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")

Name 컬럼 출력
오류 발생: Request 'The command in the required format is: 
column:Name. 

This will display the 'Name' column of the DataFrame. 

If you would like to see a specific row or a range of rows, you can modify it as follows:
- To get a specific row: row:1[Name]
- To get a column for specific rows: column:Name[1,3,5]
- To get a column for a range of rows: column:Name[1..3] 
- To perform an operation: mean:Name is not valid since mean is not a string column, but  "mean:Age" or  "sum:Fare" could be used.' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [ ]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
# model = ChatOpenAI(temperature=0, model_name="gpt-4o")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 응답 스키마 정의 {data : [{},{},{}] }
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"data": string  // A list of dictionaries representing table rows.\n}\n```'} template='\n    You are an AI assistant that generates tabular data. \n    You must return the data in JSON format that follows this schema:\n\n    {format_instructions}\n\n    **User Query:**\n    {query}\n    '


In [10]:
# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})
        print(json_response)
        
        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [11]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the second half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
print(df_seoul_housing.shape)
df_seoul_housing

2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성
{'data': [{'District': 'Gangnam-gu', 'Average Price (KRW)': 1200000000, 'Number of Transactions': 150, 'Year-over-Year Change (%)': 10.2}, {'District': 'Gangdong-gu', 'Average Price (KRW)': 900000000, 'Number of Transactions': 120, 'Year-over-Year Change (%)': 8.5}, {'District': 'Gwangjin-gu', 'Average Price (KRW)': 1000000000, 'Number of Transactions': 180, 'Year-over-Year Change (%)': 12.1}, {'District': 'Dongdaemun-gu', 'Average Price (KRW)': 800000000, 'Number of Transactions': 100, 'Year-over-Year Change (%)': 6.8}, {'District': 'Dongjak-gu', 'Average Price (KRW)': 950000000, 'Number of Transactions': 140, 'Year-over-Year Change (%)': 9.2}, {'District': 'Eunpyeong-gu', 'Average Price (KRW)': 700000000, 'Number of Transactions': 80, 'Year-over-Year Change (%)': 5.5}, {'District': 'Geumcheon-gu', 'Average Price (KRW)': 600000000, 'Number of Transactions': 60, 'Year-over-Year Change (%)': 4.2}, {'District': 'Gu-ro-gu', 'Average Price (KRW)': 850000000

,District,Average Price (KRW),Number of Transactions,Year-over-Year Change (%)
0,Gangnam-gu,1200000000,150,10.2
1,Gangdong-gu,900000000,120,8.5
2,Gwangjin-gu,1000000000,180,12.1
3,Dongdaemun-gu,800000000,100,6.8
4,Dongjak-gu,950000000,140,9.2
5,Eunpyeong-gu,700000000,80,5.5
6,Geumcheon-gu,600000000,60,4.2
7,Gu-ro-gu,850000000,110,7.5
8,Gwanak-gu,650000000,90,5.8
9,Gwangju,750000000,130,8.1


In [12]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    "Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
)
if df_seoul_subway is not None:
    #print(df_seoul_subway.shape)
    df_seoul_subway.head()

2024년 서울 지하철역별 유동 인구 데이터
{'data': [{'Station Name': 'Gangnam Station', 'Line Number': '2', 'Daily Passenger Volume': '146,321', 'Weekday vs Weekend Ratio': '1.23'}, {'Station Name': 'Hongdae Station', 'Line Number': '2', 'Daily Passenger Volume': '134,819', 'Weekday vs Weekend Ratio': '1.17'}, {'Station Name': 'Itaewon Station', 'Line Number': '4', 'Daily Passenger Volume': '123,456', 'Weekday vs Weekend Ratio': '1.05'}, {'Station Name': 'Yongsan Station', 'Line Number': '1, 4', 'Daily Passenger Volume': '114,285', 'Weekday vs Weekend Ratio': '1.22'}, {'Station Name': 'Jamsil Station', 'Line Number': '2, 8', 'Daily Passenger Volume': '107,692', 'Weekday vs Weekend Ratio': '1.11'}, {'Station Name': 'Bongeunsa Station', 'Line Number': '2', 'Daily Passenger Volume': '104,167', 'Weekday vs Weekend Ratio': '1.19'}, {'Station Name': 'Seoul Station', 'Line Number': '1, 4, Gyeongbu', 'Daily Passenger Volume': '98,654', 'Weekday vs Weekend Ratio': '1.25'}, {'Station Name': 'Dongdaemun History &

In [13]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    "Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
)
df_korean_convenience_stores.head()

한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
{'data': [{'Brand Name': 'CU', 'Number of Stores': 15432, 'Total Revenue': 11.5, 'Market Share': 27.3}, {'Brand Name': 'GS25', 'Number of Stores': 13345, 'Total Revenue': 9.8, 'Market Share': 23.5}, {'Brand Name': '7-Eleven', 'Number of Stores': 11256, 'Total Revenue': 8.2, 'Market Share': 20.2}, {'Brand Name': 'E-Mart24', 'Number of Stores': 9012, 'Total Revenue': 6.5, 'Market Share': 16.1}, {'Brand Name': 'Homeplus', 'Number of Stores': 7010, 'Total Revenue': 5.2, 'Market Share': 12.9}]}

🔹 Generated DataFrame:



,Brand Name,Number of Stores,Total Revenue,Market Share
0,CU,15432,11.5,27.3
1,GS25,13345,9.8,23.5
2,7-Eleven,11256,8.2,20.2
3,E-Mart24,9012,6.5,16.1
4,Homeplus,7010,5.2,12.9


### PydanticOutputParser

In [14]:
# 필수 라이브러리 설치
#%pip install pydantic

from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List


In [15]:

# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel): # BaseModel 상속
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"movie_title": {"description": "추천 영화 제목", "title": "Movie Title", "type": "string"}, "reason": {"description": "추천 이유", "title": "Reason", "type": "string"}, "genre": {"description": "영화 장르", "items": {"type": "string"}, "title": "Genre", "type": "array"}, "estimated_rating": {"description": "10점 만점에서 예상 평점", "title": "Estimated Rating", "type": "number"}}, "required": ["movie_title", "reason", "genre", "estimated_rating"]}\n```'} messag

In [16]:

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")
    

추천 영화: The Silence of the Lambs
추천 이유: 1990년대 클래식한 느낌의 공포 영화로, 오스카 최우수 작품상을 수상한 작품입니다.
장르: 공포, 스릴러, 미스터리
예상 평점: 9.5/10


### StructuredOutputParser

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)


출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```


In [18]:

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'cons': ['가격이 비쌈', '무게가 200g 이상임', '손이 피곤할 수 있음'],
 'pros': ['배터리 수명이 좋음', '카메라 화질이 선명함', '야간 모드가 훌륭함'],
 'rating': '4',
 'summary': '스마트폰의 배터리 수명과 카메라 화질은 우수하나, 가격이 비싸고 무게가 무거워서 손이 피곤할 수 있음.'}


### DatetimeOutputParser